In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import timescaledb_model as tsdb
from utils import get_files_infos_df, read_file, get_files_infos_windows_df
from tqdm import tqdm 
from utils import multi_read_df_from_paths
from analyze import dfs_to_stocks
from constant import DATA_PATH, IS_DOCKER
from concurrent.futures import ThreadPoolExecutor
import os

In [3]:
files_infos_df = get_files_infos_df()
# files_infos_df = get_files_infos_windows_df(files_info_path) #for windows

In [4]:
files_infos_df

,market,path,year,year_month,date,name
timestamp,,,,,,
2019-01-01 09:05:02,amsterdam,../data/boursorama/2019/amsterdam 2019-01-01 0...,2019,2019-01,2019-01-01,amsterdam 2019-01-01 09:05:02.607291.bz2
2019-01-01 09:05:02,compB,../data/boursorama/2019/compB 2019-01-01 09:05...,2019,2019-01,2019-01-01,compB 2019-01-01 09:05:02.607291.bz2
2019-01-01 09:05:02,compA,../data/boursorama/2019/compA 2019-01-01 09:05...,2019,2019-01,2019-01-01,compA 2019-01-01 09:05:02.607291.bz2
2019-01-01 09:15:02,compB,../data/boursorama/2019/compB 2019-01-01 09:15...,2019,2019-01,2019-01-01,compB 2019-01-01 09:15:02.868405.bz2
2019-01-01 09:15:02,compA,../data/boursorama/2019/compA 2019-01-01 09:15...,2019,2019-01,2019-01-01,compA 2019-01-01 09:15:02.868405.bz2
...,...,...,...,...,...,...
2023-12-29 17:42:02,compB,../data/boursorama/2023/compB 2023-12-29 17:42...,2023,2023-12,2023-12-29,compB 2023-12-29 17:42:02.037652.bz2
2023-12-29 17:52:01,compA,../data/boursorama/2023/compA 2023-12-29 17:52...,2023,2023-12,2023-12-29,compA 2023-12-29 17:52:01.887807.bz2
2023-12-29 17:52:01,peapme,../data/boursorama/2023/peapme 2023-12-29 17:5...,2023,2023-12,2023-12-29,peapme 2023-12-29 17:52:01.887807.bz2


In [5]:
np.random.seed(10)
dates_to_take = files_infos_df.groupby([files_infos_df["year_month"]]).apply(lambda x :x.sample(n=1)).reset_index(drop=True)["date"]
file_infos_df_filtered = files_infos_df[files_infos_df["date"].isin(dates_to_take)]
file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour
file_infos_df_filtered = file_infos_df_filtered.groupby([file_infos_df_filtered["date"], file_infos_df_filtered["hour"], file_infos_df_filtered["market"]]).first().reset_index()

/var/folders/lb/9z4xr9hd1013w3_cbsw1p6600000gn/T/ipykernel_35780/2876105170.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dates_to_take = files_infos_df.groupby([files_infos_df["year_month"]]).apply(lambda x :x.sample(n=1)).reset_index(drop=True)["date"]
/var/folders/lb/9z4xr9hd1013w3_cbsw1p6600000gn/T/ipykernel_35780/2876105170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour


In [6]:
file_infos_df_companies_filtered = files_infos_df.groupby(["market", "date"]).first()
file_infos_df_companies_filtered = file_infos_df_companies_filtered.reset_index()

## DB

In [21]:
import timescaledb_model as tsdb
db = (
    tsdb.TimescaleStockMarketModel("bourse", "ricou", "db", "monmdp", setup=False)
    if IS_DOCKER
    else tsdb.TimescaleStockMarketModel("bourse", "ricou", "localhost", "monmdp", setup=False)
)

Logs of timescaledb_model go to ../data/bourse.log
Connection attempt failed: connection to server at "localhost" (::1), port 5432 failed: FATAL:  the database system is not yet accepting connections
DETAIL:  Consistent recovery state has not been yet reached.

Connection attempt failed: connection to server at "localhost" (::1), port 5432 failed: FATAL:  the database system is not yet accepting connections
DETAIL:  Consistent recovery state has not been yet reached.

Connection attempt failed: connection to server at "localhost" (::1), port 5432 failed: FATAL:  the database system is not yet accepting connections
DETAIL:  Consistent recovery state has not been yet reached.

Connection attempt failed: connection to server at "localhost" (::1), port 5432 failed: FATAL:  the database system is not yet accepting connections
DETAIL:  Consistent recovery state has not been yet reached.

Connection attempt failed: connection to server at "localhost" (::1), port 5432 failed: FATAL:  the datab

Exception: Failed to connect to database after multiple attempts

# Utils

In [11]:
import re

def is_possible_ticker_with_number(symbol):
    return bool(re.match(r'^[A-Z0-9\-]{1,10}$', symbol))

def detection_intrus(df: pd.DataFrame):
    return df[(df["ticker"].apply(is_possible_ticker_with_number) == False)]

In [12]:
def get_prefix(df: pd.DataFrame, prefix_function = lambda x : x[0:3]) -> set:
    return set(df["symbol"].apply(prefix_function).unique())

In [13]:
map_prefix_to_symbol_nf = {
    "1rP": lambda x: x[3:].split("_")[0] if len(x) != 15 else np.nan,  # EuroNext Pars
    "1rA": lambda x: x[3:],  # EuroNext Amsterdam
    "1rE": lambda x: x[4:],  # EuroNext Paris
    "FF1": lambda x: x.split("_")[1][0:],  # EuroNext Brussels
}


def update_ticker_column(
    df: pd.DataFrame,
) -> pd.DataFrame:
    df["ticker"] = df.apply(
        lambda x: map_prefix_to_symbol_nf.get(x["prefix"], lambda _: x["symbol"])(
            x["symbol"]
        ),
        axis=1,
    )
    return df

def update_mid_column(
    df: pd.DataFrame,
    prefix_to_market_id: dict,
    default_mid: int,
) -> pd.DataFrame:
    df["mid"] = df.apply(
        lambda x: prefix_to_market_id.get(x["prefix"],default_mid),
        axis=1,
    )
    return df

In [14]:
def dfs_to_companie(
    dfs: list[pd.DataFrame], prefix_to_market_id: dict, default_mid: int, is_pea: bool = False
) -> pd.DataFrame:
    df_all_days = pd.concat(dfs)
    df_all_days_grouped = (
        df_all_days.reset_index(drop=True).groupby(["symbol", "name"]).last()
    )
    df_all_days_grouped["name"] = df_all_days_grouped.index.get_level_values(1)
    df_companies = df_all_days_grouped.groupby(
        df_all_days_grouped.index.get_level_values(0)
    ).last()
    df_companies = df_companies.reset_index()[["symbol", "name", "timestamp"]]
    df_companies["prefix"] = df_companies["symbol"].apply(lambda x: x[0:3])
    df_companies = update_ticker_column(df_companies)
    df_companies = update_mid_column(df_companies, prefix_to_market_id, default_mid=default_mid)
    df_companies = df_companies.drop(columns=["prefix"])
    df_companies["pea"] = is_pea
    return df_companies

## Amsterdam - Companies handling

In [23]:
dfs_amsterdam = multi_read_df_from_paths(list(file_infos_df_companies_filtered[file_infos_df_companies_filtered["market"] == "amsterdam"] ["path"]))

100%|██████████| 1255/1255 [00:21<00:00, 57.71it/s]


In [24]:
amsterdam_companies = dfs_to_companie(dfs_amsterdam, db.prefix_to_market_id, default_mid=db.nasdaq_market_id) 

In [25]:
amsterdam_companies

,symbol,name,timestamp,ticker,mid,pea
0,124718367,ASTRAZENECA FIN CVR,2023-03-27 09:02:02,124718367,14,False
1,AABA,ALTABA,2019-10-29 09:02:02,AABA,14,False
2,AACG,ATA CRTV GLB SP ADR,2023-12-29 09:02:02,AACG,14,False
3,AACI,ARMADA ACQN I,2023-12-29 09:02:02,AACI,14,False
4,AACIU,ARMADA ACQN I UTS,2023-12-29 09:02:02,AACIU,14,False
...,...,...,...,...,...,...
6747,ZWRKU,Z-WORK ACQN UTS 26,2023-01-02 09:02:01,ZWRKU,14,False
6748,ZY,ZYMERGEN,2022-11-18 09:02:02,ZY,14,False
6749,ZYME,ZYMEWORKS,2023-12-29 09:02:02,ZYME,14,False
6750,ZYNE,ZYNERBA PHARMA,2023-11-03 09:02:01,ZYNE,14,False


In [15]:
# detection_intrus(amsterdam_companies)

## CompA - Companies handling

In [27]:
dfs_compA = multi_read_df_from_paths(list(file_infos_df_companies_filtered[file_infos_df_companies_filtered["market"] == "compA"] ["path"]))

100%|██████████| 1264/1264 [00:05<00:00, 211.11it/s]


In [28]:
compA_companies = dfs_to_companie(dfs_compA, prefix_to_market_id=db.prefix_to_market_id ,default_mid=db.prefix_to_market_id["1rP"])

In [29]:
compA_companies

,symbol,name,timestamp,ticker,mid,pea
0,1rAAF,AIR FRANCE - KLM,2023-12-29 09:02:02,AF,11,False
1,1rAAFA,AIR FRANCE - KLM,2023-12-29 09:02:02,AFA,11,False
2,1rAENX,EURONEXT,2023-12-29 09:02:02,ENX,11,False
3,1rAGTO,GEMALTO,2019-07-11 09:02:02,GTO,11,False
4,1rARDSADT,ROYAL D SH DRP 21,2021-11-15 09:02:01,RDSADT,11,False
...,...,...,...,...,...,...
416,FF11_FP,TOTALENERGIES,2021-08-06 09:02:01,FP,13,False
417,FF11_SEV,SUEZ,2023-12-29 09:02:02,SEV,13,False
418,FF11_SGO,SAINT-GOBAIN,2023-12-29 09:02:02,SGO,13,False
419,FF11_TTE,TOTALENERGIES,2023-12-29 09:02:02,TTE,13,False


In [121]:
# detection_intrus(compA_companies)

In [30]:
get_prefix(compA_companies)

{'1rA', '1rP', 'ALF', 'FF1', 'INE'}

## CompB - Companies handling

In [32]:
dfs_compB = multi_read_df_from_paths(list(file_infos_df_companies_filtered[file_infos_df_companies_filtered["market"] == "compB"] ["path"]))

100%|██████████| 1263/1263 [00:05<00:00, 238.41it/s]


In [33]:
compB_companies = dfs_to_companie(dfs_compB, prefix_to_market_id=db.prefix_to_market_id ,default_mid=db.prefix_to_market_id["1rP"])

In [34]:
compB_companies

,symbol,name,timestamp,ticker,mid,pea
0,1rP03227,LATECOERE,2023-12-29 09:02:02,03227,12,False
1,1rP2MX,TERACT R PFD,2023-12-29 09:02:02,2MX,12,False
2,1rP5478,CLARANOVA,2021-11-08 09:02:01,5478,12,False
3,1rPAAA,ALAN ALLMAN ASSOCIATES,2023-12-29 09:02:02,AAA,12,False
4,1rPAAC,ACCOR ACQ COM R PFD,2021-06-07 09:02:01,AAC,12,False
...,...,...,...,...,...,...
391,1rPXFAB,X-FAB SILICON,2023-12-29 09:02:02,XFAB,12,False
392,1rPXIL,XILAM ANIMATION,2023-12-29 09:02:02,XIL,12,False
393,1rPYSYT,SALVEPORN01JAN22EX,2023-12-29 09:02:02,YSYT,12,False
394,FF11_AKA,AKKA TECHNOLOGIES,2022-02-01 09:02:01,AKA,13,False


In [35]:
get_prefix(compB_companies)

{'1rP', 'FF1'}

## PEA PME - Companies Handling

In [36]:
dfs_peapme = multi_read_df_from_paths(list(file_infos_df_companies_filtered[file_infos_df_companies_filtered["market"] == "peapme"] ["path"]))

100%|██████████| 766/766 [00:03<00:00, 201.57it/s]


In [37]:
peapme_companies = dfs_to_companie(dfs_peapme, prefix_to_market_id=db.prefix_to_market_id ,default_mid=db.prefix_to_market_id["1rP"], is_pea=True)

In [38]:
peapme_companies

,symbol,name,timestamp,ticker,mid,pea
0,1rAADUX,ADUX,2023-12-29 09:02:02,ADUX,11,True
1,1rABESI,BESI,2023-12-29 09:02:02,BESI,11,True
2,1rEPALANT,ANTEVENIO,2021-12-01 09:02:01,ALANT,12,True
3,1rEPALAQU,AQUILA,2023-12-29 09:02:02,ALAQU,12,True
4,1rEPALBDM,BD MULTI MEDIA,2023-12-29 09:02:02,ALBDM,12,True
...,...,...,...,...,...,...
678,FF11_EVS,EVS BROADCAST EQU,2023-12-29 09:02:02,EVS,13,True
679,FF11_GKTX,GENKYOTEX,2022-09-15 09:02:02,GKTX,13,True
680,FF11_KIN,KINEPOLIS GROUP,2023-12-29 09:02:02,KIN,13,True
681,FF11_MLMAZ,MAZARO,2023-12-29 09:02:02,MLMAZ,13,True


In [39]:
get_prefix(peapme_companies)

{'1rA', '1rE', '1rP', 'FF1'}

## Join companies

In [40]:
df_companies = [amsterdam_companies, compA_companies, compB_companies, peapme_companies]

In [41]:
df_companies = pd.concat(df_companies)
df_companies.sort_values(by="timestamp", inplace=True)
df_companies.drop_duplicates(inplace=True)
df_companies.drop_duplicates(subset=["symbol"], keep="last", inplace=True)

### Check no duplicated

In [42]:
df_companies[df_companies.duplicated(subset=["symbol"], keep=False)][:50]

,symbol,name,timestamp,ticker,mid,pea


In [43]:
df_companies.drop("timestamp", axis=1, inplace=True)

In [44]:
df_companies.set_index("symbol", inplace=True)

In [45]:
df_companies.head()

,name,ticker,mid,pea
symbol,,,,
ARII,AMERICAN RAILCAR,ARII,14,False
INDU,INDUSTR ACQST-A-,INDU,14,False
INDUU,INDUSTREA ACQST UTS,INDUU,14,False
JTPY,JETPAY,JTPY,14,False
MMAC,MMA CAPITAL MANA,MMAC,14,False


### Update companies table

In [137]:
# db.clean_database()
# db._setup_database()

In [46]:
db.df_write(df_companies, "companies", commit=True)

# Stocks

In [8]:
dfs = multi_read_df_from_paths(list(file_infos_df_filtered ["path"]))

  0%|          | 0/1944 [00:00<?, ?it/s]

100%|██████████| 1944/1944 [00:20<00:00, 93.25it/s] 


In [9]:
df_stocks = dfs_to_stocks(dfs) 

0


In [20]:
df_stocks

,level_0,symbol,timestamp,index,last,volume
909,909,1rABESI,2021-01-01 09:02:00,825957.0,49.58,140312.0
910,910,1rABESI,2021-01-01 10:02:00,830626.0,49.58,140312.0
911,911,1rABESI,2021-01-01 11:02:00,835294.0,49.58,140312.0
912,912,1rABESI,2021-01-01 12:02:00,839963.0,49.58,140312.0
913,913,1rABESI,2021-01-01 13:02:00,844632.0,49.58,140312.0
...,...,...,...,...,...,...
2585943,2585943,ZYXI,2022-02-21 13:02:00,1467238.0,6.76,26149.0
2585944,2585944,ZYXI,2022-02-21 14:02:00,1473149.0,6.76,26149.0
2585945,2585945,ZYXI,2022-02-21 15:02:00,1479060.0,6.76,26149.0
2585946,2585946,ZYXI,2022-02-21 16:02:00,1484971.0,6.76,26149.0


#### Show Nan 

In [10]:
df_stocks[df_stocks['last'].isna()], df_stocks[df_stocks['volume'].isna()]

(Empty DataFrame
 Columns: [last, volume, timestamp]
 Index: [],
 Empty DataFrame
 Columns: [last, volume, timestamp]
 Index: [])

#### Show negative values 

In [11]:
df_stocks[df_stocks['volume'] < 0], df_stocks[df_stocks['last'] < 0]

(         last  volume           timestamp
 symbol                                   
 1rPCDA  16.60  -12475 2020-04-07 12:02:02
 1rPCDA  16.72  -10871 2020-04-07 13:02:02
 1rPCDA  16.94   -9815 2020-04-07 14:02:02
 1rPCDA  16.68   -9298 2020-04-07 15:02:01
 1rPCDA  16.74   -9280 2020-04-07 16:02:01
 1rPCDA  16.88   -8366 2020-04-07 17:02:02,
 Empty DataFrame
 Columns: [last, volume, timestamp]
 Index: [])

In [12]:
df_stocks.reset_index(inplace=True)
df_stocks.set_index("timestamp", inplace=True)

In [13]:
pcda = df_stocks[df_stocks["symbol"]=="1rPCDA" ]
pcda.loc["2020-04-07"]

,symbol,last,volume
timestamp,,,
2020-04-07 09:02:01,1rPCDA,16.12,876
2020-04-07 10:02:01,1rPCDA,16.36,2108
2020-04-07 11:02:01,1rPCDA,16.40,4414
2020-04-07 12:02:02,1rPCDA,16.60,-12475
2020-04-07 13:02:02,1rPCDA,16.72,-10871
2020-04-07 14:02:02,1rPCDA,16.94,-9815
2020-04-07 15:02:01,1rPCDA,16.68,-9298
2020-04-07 16:02:01,1rPCDA,16.74,-9280
2020-04-07 17:02:02,1rPCDA,16.88,-8366


#### Show zero values for volume

In [14]:
df_stocks[df_stocks['volume'] == 0]

,symbol,last,volume
timestamp,,,
2019-01-15 09:02:02,ABII,78.25,0
2019-01-15 09:02:02,ACCL,12.51,0
2019-01-15 09:02:02,ACTL,20.95,0
2019-01-15 09:02:02,ADGF,10.78,0
2019-01-15 09:02:02,AGAM,21.20,0
...,...,...,...
2023-12-21 17:02:02,1rPMLUMG,2.40,0
2023-12-21 17:02:02,1rPALUPG,1.80,0
2023-12-21 17:02:02,1rPMLVAL,14.30,0


#### Show negatives values for last

In [15]:
df_stocks[df_stocks['last'] < 0]

,symbol,last,volume
timestamp,,,


#### Show zero values for last

In [16]:
df_stocks[df_stocks['last'] == 0]

,symbol,last,volume
timestamp,,,
2019-01-15 09:02:02,ALZH,0.0,0
2019-01-15 09:02:02,AMCI,0.0,0
2019-01-15 09:02:02,ZLIG,0.0,0
2019-01-15 09:02:02,ASBC,0.0,0
2019-01-15 09:02:02,BECM,0.0,0
...,...,...,...
2023-12-21 17:02:02,1rPALGEC,0.0,14464536
2023-12-21 17:02:02,1rPMLGLA,0.0,0
2023-12-21 17:02:02,1rP03227,0.0,0


#### Show max and min values for last

In [17]:
df_stocks[df_stocks['last'] == df_stocks['last'].max()].head(1)

,symbol,last,volume
timestamp,,,
2019-01-15 09:02:02,1rPMALA,8750.0,0


In [18]:
df_stocks[df_stocks['last'] == df_stocks['last'].min()].head(1)

,symbol,last,volume
timestamp,,,
2019-01-15 09:02:02,ALZH,0.0,0


#### Show the max and min value of the volume 

## Write To Database

In [12]:
symbol_to_companies = {
        v: k for k, v in dict(db.raw_query("SELECT id, symbol FROM companies")).items()
    }

### Update Stocks (OLD WAY)

In [13]:
df_stocks["cid"] = df_stocks["symbol"].apply(
    lambda x: symbol_to_companies.get(x, None)
)
df_stocks.rename(columns={"timestamp": "date", "last": "value"}, inplace=True)
df_stocks = df_stocks[["date", "cid", "value", "volume"]]

In [14]:
df_stocks["cid"] = df_stocks['cid'].astype(np.int16)
df_stocks['value'] = df_stocks['value'].astype(np.float32)
df_stocks['volume'] = df_stocks['volume'].astype(np.int64)

#### Bottleneck

In [51]:
db.df_write(df_stocks, "stocks", index=False)

#### NEW WAY

In [16]:
chunks = np.array_split(df_stocks, 8)

In [ ]:
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(db.copy_from_stringio, chunk, "stocks") for chunk in chunks]
    for future in futures:
        future.result() 

In [87]:
db.copy_from_stringio(df_stocks, "stocks")

# Days Stocks

In [92]:
from datetime import datetime, timedelta, date

dates_group = [
    datetime.date(datetime(2019, 1, 1)),
    datetime.date(datetime(2019, 1, 2)),
]

In [114]:
def update_daystocks(db, dates_group: list[date]):
  db.execute(
        f"""
  INSERT INTO daystocks (date, cid, open, close, high, low, volume)
  SELECT
  DATE(date) AS date,
  cid,
  FIRST(value, date) AS open,
  LAST(value, date) AS close,
  MAX(value) AS high,
  MIN(value) AS low,
  LAST(volume, date) AS volume
  FROM
  stocks
  WHERE
  date > '{dates_group[0]}' and date < '{dates_group[1] +timedelta(days=1)}'
  GROUP BY
    cid,
    DATE(date)
  ORDER BY
    DATE(date) DESC
  """, commit = True
    )

In [115]:
update_daystocks(db, dates_group)

start
commit


## Reset File done

##### Select just 2023 december files

In [ ]:
db.raw_query(
    """SELECT name
FROM file_done
WHERE EXTRACT(MONTH FROM TO_TIMESTAMP(RIGHT(name, 30), 'YYYY-MM-DD HH24:MI:SS')) = 12
AND EXTRACT(YEAR FROM TO_TIMESTAMP(RIGHT(name, 30), 'YYYY-MM-DD HH24:MI:SS')) = 2023
ORDER BY name DESC"""
)

## Python script

In [51]:
db.clean_database()
db._setup_database()